In [ ]:
import os
from langchain.chat_models import init_chat_model

# 设置自定义API配置
os.environ["QWEN_API_KEY"] = "You API Key"
os.environ["QWEN_API_BASE"] = "https://dashscope.aliyuncs.com/compatible-mode/v1"

# 使用自定义配置
llm = init_chat_model(
    model="qwen-plus-latest",
    model_provider="openai",
    api_key=os.environ["QWEN_API_KEY"],
    base_url=os.environ["QWEN_API_BASE"]
)

In [2]:
import os
import getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("TAVILY_API_KEY")

TAVILY_API_KEY:  ········


In [4]:
import operator
from typing import Annotated, List, TypedDict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.messages import SystemMessage, HumanMessage

# ---------- 1. 全局 LLM & 工具 ----------
#llm = ChatOpenAI(model="gpt-4o", temperature=0.7)  # 修复：取消注释
search = TavilySearchResults(max_results=3)

# ---------- 2. 数据结构 ----------
class Section(BaseModel):
    title: str = Field(description="Section title")
    description: str = Field(description="Section description")

class Plan(BaseModel):
    sections: List[Section] = Field(description="List of report sections")

class State(TypedDict):
    topic: str
    plan: List[Section]
    drafts: Annotated[List[str], operator.add]
    final_report: str

class WorkerState(TypedDict):
    section: Section
    drafts: Annotated[List[str], operator.add]  # 修复：匹配状态键

# ---------- 3. Plan-and-Solve：生成大纲 ----------
planner = llm.with_structured_output(Plan)

def plan_node(state: State):
    # 修复：添加 JSON 关键词
    prompt = f"""为"{state['topic']}"写一份详细的大纲，每节含标题和 20 字描述。
    
    请以 JSON 格式返回，包含以下结构：
    {{
        "sections": [
            {{"title": "章节标题", "description": "章节描述"}}
        ]
    }}
    """
    
    plan = planner.invoke([
        SystemMessage(content="你是一个专业的报告规划师，请按照 JSON 格式返回报告大纲。"),
        HumanMessage(content=prompt)
    ])
    return {"plan": plan.sections}

# ---------- 4. ReAct：联网搜集 ----------
def research(section: Section) -> str:
    try:
        query = f"{section.title} {section.description}"
        docs = search.invoke(query)
        return "\n".join([d["content"][:500] for d in docs])
    except Exception as e:
        print(f"搜索错误: {e}")
        return f"无法获取 {section.title} 的相关信息"

# ---------- 5. Worker：写作 + Reflection ----------
writer_prompt = """你是一名专栏作家。
标题：{title}
描述：{desc}
素材：{material}
请用 Markdown 写一篇 400 字左右的正文，不要前言。
"""

critic_prompt = """请对以下文字提出 3 条具体改进意见：
{text}
"""

def worker_node(state: WorkerState):
    print(f"🔄 处理章节: {state['section'].title}")
    
    material = research(state['section'])
    
    # 初稿
    draft1 = llm.invoke(writer_prompt.format(
        title=state['section'].title,
        desc=state['section'].description,
        material=material
    )).content
    
    # 反思
    critique = llm.invoke(critic_prompt.format(text=draft1)).content
    
    # 重写
    draft2 = llm.invoke(
        f"根据意见重写：\n意见：{critique}\n原文：{draft1}"
    ).content
    
    print(f"✅ 完成章节: {state['section'].title}")
    return {"drafts": [draft2]}  # 修复：返回 drafts 而不是 draft

# ---------- 6. Map-Reduce：并行分发 ----------
def dispatch(state: State):
    print(f"📋 规划完成，共 {len(state['plan'])} 个章节")
    return [Send("worker", {"section": s}) for s in state["plan"]]

# ---------- 7. 合并节点 ----------
def merge(state: State):
    print(f"📝 合并 {len(state['drafts'])} 个章节")
    full = f"# {state['topic']}\n\n"
    full += "\n\n---\n\n".join(state["drafts"])
    return {"final_report": full}

# ---------- 8. 构建图 ----------
graph_builder = StateGraph(State)
graph_builder.add_node("plan", plan_node)
graph_builder.add_node("worker", worker_node)
graph_builder.add_node("merge", merge)

graph_builder.add_edge(START, "plan")
graph_builder.add_conditional_edges("plan", dispatch, ["worker"])
graph_builder.add_edge("worker", "merge")
graph_builder.add_edge("merge", END)

app = graph_builder.compile()

# ---------- 9. 运行 ----------
if __name__ == "__main__":
    print("🚀 开始生成报告...")
    topic = "生成式 AI 如何重塑教育"
    result = app.invoke({"topic": topic})
    print("\n" + "="*50)
    print("📊 最终报告:")
    print("="*50)
    print(result["final_report"])  # 修复：去掉多余的冒号

🚀 开始生成报告...
📋 规划完成，共 10 个章节
🔄 处理章节: 引言：AI 技术的发展与教育变革
🔄 处理章节: 个性化学习的实现路径
🔄 处理章节: 生成式 AI 的基本原理与功能
🔄 处理章节: 教学内容的自动化生成与优化
🔄 处理章节: 智能辅导与学习评估系统
🔄 处理章节: 教师角色的转变与能力提升
🔄 处理章节: 教育公平与技术普及的挑战
🔄 处理章节: 伦理与隐私保护问题探讨
🔄 处理章节: 未来教育生态的重构与展望
🔄 处理章节: 政策支持与教育 AI 的健康发展
✅ 完成章节: 伦理与隐私保护问题探讨
✅ 完成章节: 引言：AI 技术的发展与教育变革
✅ 完成章节: 教学内容的自动化生成与优化
✅ 完成章节: 未来教育生态的重构与展望
✅ 完成章节: 个性化学习的实现路径
✅ 完成章节: 教师角色的转变与能力提升
✅ 完成章节: 智能辅导与学习评估系统
✅ 完成章节: 生成式 AI 的基本原理与功能
✅ 完成章节: 教育公平与技术普及的挑战
✅ 完成章节: 政策支持与教育 AI 的健康发展
📝 合并 10 个章节

📊 最终报告:
# 生成式 AI 如何重塑教育

根据您的修改意见，我对原文进行了优化，增强了段落间的逻辑衔接、提升了语言的精准度，并补充了具体案例以增强说服力。以下是修改后的版本：

---

## AI 技术的发展与教育变革  

人工智能技术的飞速发展正在深刻影响教育的各个方面。从智能辅导系统到个性化学习路径，AI 不仅改变了传统教学模式，还为学生提供更具针对性的学习体验。通过分析学习数据，AI 能够识别个体差异，自动调整教学内容和节奏，从而提升学习效率，弥补传统课堂中“一刀切”教学方式的不足。例如，某些智能辅导平台已能够根据学生的答题记录实时调整题目难度与讲解策略，实现真正意义上的因材施教。

尽管 AI 在教学中展现出巨大潜力，其广泛应用仍面临一系列现实挑战。在技术层面，当前的 AI 系统在面对复杂的教育场景时，仍存在适应性不足的问题。学习数据的稀疏性和算法模型的局限性，可能导致个性化推荐出现偏差，从而影响教学效果。此外，随着 AI 在教学中的深入应用，教师的角色也发生了变化。教育者不再只是知识的传授者，而需转变为学习的引导者和情感的支持者。如何在技术辅助下保持人文关怀，是教育工作者必须思考的重要课题。

在此基础上，A

In [5]:
import operator
from typing import Annotated, List, TypedDict, Dict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.messages import SystemMessage, HumanMessage

# ---------- 1. 全局 LLM & 工具 ----------
#llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
search = TavilySearchResults(max_results=3)

# ---------- 2. 数据结构 ----------
class Section(BaseModel):
    title: str = Field(description="Section title")
    description: str = Field(description="Section description")

class Plan(BaseModel):
    sections: List[Section] = Field(description="List of report sections")

# 修改：添加带索引的章节结果
class IndexedDraft(BaseModel):
    index: int
    content: str

class State(TypedDict):
    topic: str
    plan: List[Section]
    drafts: Annotated[List[IndexedDraft], operator.add]  # 修改：使用带索引的结果
    final_report: str

class WorkerState(TypedDict):
    section: Section
    index: int  # 修改：添加索引
    drafts: Annotated[List[IndexedDraft], operator.add]

# ---------- 3. Plan-and-Solve：生成大纲 ----------
planner = llm.with_structured_output(Plan)

def plan_node(state: State):
    prompt = f"""为"{state['topic']}"写一份详细的大纲，每节含标题和 20 字描述。
    
    请以 JSON 格式返回，包含以下结构：
    {{
        "sections": [
            {{"title": "章节标题", "description": "章节描述"}}
        ]
    }}
    """
    
    plan = planner.invoke([
        SystemMessage(content="你是一个专业的报告规划师，请按照 JSON 格式返回报告大纲。"),
        HumanMessage(content=prompt)
    ])
    return {"plan": plan.sections}

# ---------- 4. ReAct：联网搜集 ----------
def research(section: Section) -> str:
    try:
        query = f"{section.title} {section.description}"
        docs = search.invoke(query)
        return "\n".join([d["content"][:500] for d in docs])
    except Exception as e:
        print(f"搜索错误: {e}")
        return f"无法获取 {section.title} 的相关信息"

# ---------- 5. Worker：写作 + Reflection ----------
writer_prompt = """你是一名专栏作家。
标题：{title}
描述：{desc}
素材：{material}
请用 Markdown 写一篇 400 字左右的正文，不要前言。
"""

critic_prompt = """请对以下文字提出 3 条具体改进意见：
{text}
"""

def worker_node(state: WorkerState):
    print(f"🔄 处理章节 {state['index'] + 1}: {state['section'].title}")
    
    material = research(state['section'])
    
    # 初稿
    draft1 = llm.invoke(writer_prompt.format(
        title=state['section'].title,
        desc=state['section'].description,
        material=material
    )).content
    
    # 反思
    critique = llm.invoke(critic_prompt.format(text=draft1)).content
    
    # 重写
    draft2 = llm.invoke(
        f"根据意见重写：\n意见：{critique}\n原文：{draft1}"
    ).content
    
    print(f"✅ 完成章节 {state['index'] + 1}: {state['section'].title}")
    
    # 修改：返回带索引的结果
    indexed_draft = IndexedDraft(index=state['index'], content=draft2)
    return {"drafts": [indexed_draft]}

# ---------- 6. Map-Reduce：并行分发 ----------
def dispatch(state: State):
    print(f"📋 规划完成，共 {len(state['plan'])} 个章节")
    # 修改：传递索引信息
    return [Send("worker", {"section": s, "index": i}) for i, s in enumerate(state["plan"])]

# ---------- 7. 合并节点 ----------
def merge(state: State):
    print(f"📝 合并 {len(state['drafts'])} 个章节")
    
    # 修改：按索引排序
    sorted_drafts = sorted(state["drafts"], key=lambda x: x.index)
    
    full = f"# {state['topic']}\n\n"
    
    # 按顺序组装内容
    for i, draft in enumerate(sorted_drafts):
        full += f"## {state['plan'][draft.index].title}\n\n"
        full += draft.content
        if i < len(sorted_drafts) - 1:
            full += "\n\n---\n\n"
    
    return {"final_report": full}

# ---------- 8. 构建图 ----------
graph_builder = StateGraph(State)
graph_builder.add_node("plan", plan_node)
graph_builder.add_node("worker", worker_node)
graph_builder.add_node("merge", merge)

graph_builder.add_edge(START, "plan")
graph_builder.add_conditional_edges("plan", dispatch, ["worker"])
graph_builder.add_edge("worker", "merge")
graph_builder.add_edge("merge", END)

app = graph_builder.compile()

# ---------- 9. 运行 ----------
if __name__ == "__main__":
    print("🚀 开始生成报告...")
    topic = "生成式 AI 如何重塑教育"
    result = app.invoke({"topic": topic})
    print("\n" + "="*50)
    print("📊 最终报告:")
    print("="*50)
    print(result["final_report"])

🚀 开始生成报告...
📋 规划完成，共 10 个章节
🔄 处理章节 2: 生成式 AI 在教育中的应用
🔄 处理章节 1: 引言：生成式 AI 的兴起
🔄 处理章节 3: 个性化学习的实现
🔄 处理章节 5: 教师角色的转变
🔄 处理章节 6: 学生能力的培养
🔄 处理章节 4: 教育资源的普及与公平
🔄 处理章节 7: 数据隐私与伦理问题
🔄 处理章节 8: 技术局限与挑战
🔄 处理章节 9: 未来教育模式的展望
🔄 处理章节 10: 结论与建议
✅ 完成章节 9: 未来教育模式的展望
✅ 完成章节 2: 生成式 AI 在教育中的应用
✅ 完成章节 3: 个性化学习的实现
✅ 完成章节 5: 教师角色的转变
✅ 完成章节 10: 结论与建议
✅ 完成章节 7: 数据隐私与伦理问题
✅ 完成章节 1: 引言：生成式 AI 的兴起
✅ 完成章节 4: 教育资源的普及与公平
✅ 完成章节 6: 学生能力的培养
✅ 完成章节 8: 技术局限与挑战
📝 合并 10 个章节

📊 最终报告:
# 生成式 AI 如何重塑教育

## 引言：生成式 AI 的兴起

以下是根据您的意见对原文进行修改后的版本，重点优化了段落间的逻辑衔接、语言表达的精炼性以及术语解释的可读性：

---

## 生成式 AI 的源起与发展

生成式人工智能是一类能够主动创造内容的人工智能技术，涵盖文本、图像、音频、视频等多种形式。它通过学习已有数据的模式与结构，生成全新的内容，展现出强大的创造力。其核心价值在于“生成”——不仅限于识别或分类，而是具备创新的能力，为人类社会带来前所未有的可能性。

生成式 AI 的历史可以追溯到 1966 年，由 MIT 教授 Joseph Weizenbaum 开发的聊天机器人 Eliza 是早期探索之一。尽管受限于当时的计算能力和数据规模，Eliza 的表现较为有限，但它为后来的对话系统奠定了基础。真正推动这一领域发展的，是深度学习技术的兴起。1986 年，Geoffrey Hinton 提出反向传播算法，这是一种让神经网络通过误差反馈不断调整参数、提升性能的方法，为后续生成模型的发展提供了关键技术支撑。

随着深度学习的不断演进，2014 年生成对抗网络（GAN）的提出成为生成式 AI 发展的重要里程碑。GAN 通过两个神经网络相互博弈的方式，使 AI 能够生成

In [6]:
import operator
import os
import json
import datetime
from typing import Annotated, List, TypedDict, Dict, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.messages import SystemMessage, HumanMessage

# ---------- 1. 全局 LLM & 工具 ----------
#llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
search = TavilySearchResults(max_results=3)

# ---------- 2. 创建工作目录 ----------
def setup_directories():
    """创建必要的工作目录"""
    dirs = ["search_results", "drafts", "reports"]
    for dir_name in dirs:
        os.makedirs(dir_name, exist_ok=True)

# ---------- 3. 数据结构 ----------
class Section(BaseModel):
    title: str = Field(description="Section title")
    description: str = Field(description="Section description")

class Plan(BaseModel):
    sections: List[Section] = Field(description="List of report sections")

class SearchResult(BaseModel):
    section_index: int
    original_query: str
    optimized_query: str
    results: List[Dict]
    processed_content: str

class IndexedDraft(BaseModel):
    index: int
    title: str
    content: str
    search_used: bool = False

class ValidationResult(BaseModel):
    is_valid: bool
    issues: List[str]
    suggestions: List[str]

class State(TypedDict):
    topic: str
    plan: List[Section]
    todo_list: List[str]
    completed_tasks: List[str]
    search_results: Annotated[List[SearchResult], operator.add]
    drafts: Annotated[List[IndexedDraft], operator.add]
    final_report: str
    validation_result: Optional[ValidationResult]
    output_format: str  # "markdown", "html", "pdf"

class WorkerState(TypedDict):
    section: Section
    index: int
    drafts: Annotated[List[IndexedDraft], operator.add]
    search_results: Annotated[List[SearchResult], operator.add]

# ---------- 4. 任务管理 ----------
def update_todo_list(todo_list: List[str], completed_tasks: List[str]):
    """更新TODO列表"""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open("todo.txt", "w", encoding="utf-8") as f:
        f.write(f"=== 报告生成任务列表 ({timestamp}) ===\n\n")
        
        f.write("## 已完成任务:\n")
        for task in completed_tasks:
            f.write(f"✅ {task}\n")
        
        f.write("\n## 待完成任务:\n")
        for task in todo_list:
            f.write(f"🔘 {task}\n")
        
        f.write(f"\n## 进度: {len(completed_tasks)}/{len(completed_tasks) + len(todo_list)}\n")

# ---------- 5. 搜索优化 ----------
def optimize_search_query(section: Section, topic: str) -> str:
    """优化搜索查询"""
    optimization_prompt = f"""
    主题: {topic}
    章节标题: {section.title}
    章节描述: {section.description}
    
    请为这个章节生成一个优化的搜索查询，要求：
    1. 包含关键词和相关术语
    2. 使用布尔运算符或引号提高精确度
    3. 考虑时效性和权威性
    4. 限制在50字以内
    
    直接返回优化后的搜索查询，不要其他解释。
    """
    
    optimized_query = llm.invoke([
        SystemMessage(content="你是搜索专家，专门优化搜索查询以获得最佳结果。"),
        HumanMessage(content=optimization_prompt)
    ]).content.strip()
    
    return optimized_query

# ---------- 6. 搜索和内容处理 ----------
def enhanced_research(section: Section, index: int, topic: str) -> SearchResult:
    """增强的搜索和内容处理"""
    try:
        # 优化搜索查询
        original_query = f"{section.title} {section.description}"
        optimized_query = optimize_search_query(section, topic)
        
        print(f"🔍 搜索查询优化: {original_query} -> {optimized_query}")
        
        # 执行搜索
        docs = search.invoke(optimized_query)
        
        # 处理搜索结果
        processing_prompt = f"""
        章节: {section.title}
        搜索结果: {json.dumps(docs, ensure_ascii=False, indent=2)}
        
        请整合这些搜索结果，要求：
        1. 提取关键信息和数据
        2. 去除重复内容
        3. 按重要性排序
        4. 保持事实准确性
        5. 整合成连贯的素材
        
        返回整合后的内容，用于章节写作。
        """
        
        processed_content = llm.invoke([
            SystemMessage(content="你是信息整合专家，擅长从多个来源提取和整合关键信息。"),
            HumanMessage(content=processing_prompt)
        ]).content
        
        # 保存搜索结果
        search_result = SearchResult(
            section_index=index,
            original_query=original_query,
            optimized_query=optimized_query,
            results=docs,
            processed_content=processed_content
        )
        
        # 保存到文件
        filename = f"search_results/section_{index:02d}_{section.title.replace(' ', '_')}.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
        
        return search_result
        
    except Exception as e:
        print(f"搜索错误: {e}")
        return SearchResult(
            section_index=index,
            original_query=original_query,
            optimized_query=optimized_query,
            results=[],
            processed_content=f"无法获取 {section.title} 的相关信息"
        )

# ---------- 7. 计划节点 ----------
planner = llm.with_structured_output(Plan)

def plan_node(state: State):
    print("📋 开始制定计划...")
    
    prompt = f"""为"{state['topic']}"写一份详细的大纲，每节含标题和 20 字描述。
    
    请以 JSON 格式返回，包含以下结构：
    {{
        "sections": [
            {{"title": "章节标题", "description": "章节描述"}}
        ]
    }}
    """
    
    plan = planner.invoke([
        SystemMessage(content="你是一个专业的报告规划师，请按照 JSON 格式返回报告大纲。"),
        HumanMessage(content=prompt)
    ])
    
    # 创建TODO列表
    todo_list = [f"生成章节: {section.title}" for section in plan.sections]
    todo_list.append("合并所有章节")
    todo_list.append("验证最终报告")
    
    update_todo_list(todo_list, [])
    
    return {
        "plan": plan.sections,
        "todo_list": todo_list,
        "completed_tasks": ["制定报告大纲"]
    }

# ---------- 8. Worker节点 ----------
def worker_node(state: WorkerState):
    section = state['section']
    index = state['index']
    
    print(f"🔄 处理章节 {index + 1}: {section.title}")
    
    # 搜索和处理信息
    search_result = enhanced_research(section, index, state.get('topic', ''))
    
    # 写作提示
    writer_prompt = f"""
    你是一名专业的报告写作专家。
    
    章节标题: {section.title}
    章节描述: {section.description}
    
    处理后的素材:
    {search_result.processed_content}
    
    请写一篇高质量的章节内容，要求：
    1. 使用 Markdown 格式
    2. 长度约 400-600 字
    3. 结构清晰，逻辑严谨
    4. 包含具体例子和数据
    5. 语言专业且易懂
    6. 为后续章节做好过渡
    
    直接返回章节内容，不要标题。
    """
    
    # 生成初稿
    draft1 = llm.invoke([
        SystemMessage(content="你是专业的报告写作专家，擅长创作高质量的技术报告。"),
        HumanMessage(content=writer_prompt)
    ]).content
    
    # 反思和改进
    critique_prompt = f"""
    请对以下章节内容进行专业评估并提出改进建议：
    
    {draft1}
    
    请从以下方面评估：
    1. 内容准确性和深度
    2. 逻辑结构和流畅性
    3. 语言表达和专业性
    4. 与主题的相关性
    5. 过渡和衔接性
    
    提供3-5条具体的改进建议。
    """
    
    critique = llm.invoke([
        SystemMessage(content="你是专业的内容评估专家，擅长提供建设性的改进建议。"),
        HumanMessage(content=critique_prompt)
    ]).content
    
    # 重写改进
    rewrite_prompt = f"""
    根据以下改进建议，重写章节内容：
    
    原文：
    {draft1}
    
    改进建议：
    {critique}
    
    请重写出改进后的版本，保持原有结构但提升质量。
    """
    
    final_draft = llm.invoke([
        SystemMessage(content="你是专业的内容编辑，擅长根据反馈改进文章质量。"),
        HumanMessage(content=rewrite_prompt)
    ]).content
    
    # 保存草稿
    draft_filename = f"drafts/section_{index:02d}_{section.title.replace(' ', '_')}.md"
    with open(draft_filename, "w", encoding="utf-8") as f:
        f.write(f"# {section.title}\n\n{final_draft}")
    
    print(f"✅ 完成章节 {index + 1}: {section.title}")
    
    # 创建索引草稿
    indexed_draft = IndexedDraft(
        index=index,
        title=section.title,
        content=final_draft,
        search_used=True
    )
    
    return {
        "drafts": [indexed_draft],
        "search_results": [search_result]
    }

# ---------- 9. 分发节点 ----------
def dispatch(state: State):
    print(f"📋 开始并行处理 {len(state['plan'])} 个章节")
    
    # 更新TODO
    completed = state['completed_tasks'] + ["制定执行计划"]
    remaining = [task for task in state['todo_list'] if task not in completed]
    update_todo_list(remaining, completed)
    
    return [Send("worker", {"section": s, "index": i, "topic": state['topic']}) 
            for i, s in enumerate(state["plan"])]

# ---------- 10. 合并和格式化 ----------
def merge_and_format(state: State):
    print(f"📝 合并 {len(state['drafts'])} 个章节")
    
    # 按索引排序
    sorted_drafts = sorted(state["drafts"], key=lambda x: x.index)
    
    # 生成过渡内容
    transition_prompt = f"""
    主题: {state['topic']}
    章节列表: {[draft.title for draft in sorted_drafts]}
    
    为这些章节生成自然的过渡句和连接词，确保整体文章的连贯性。
    请为每个章节之间提供1-2句过渡语。
    
    返回格式：
    章节1 -> 章节2: [过渡语]
    章节2 -> 章节3: [过渡语]
    ...
    """
    
    transitions = llm.invoke([
        SystemMessage(content="你是专业的文章编辑，擅长创建流畅的章节过渡。"),
        HumanMessage(content=transition_prompt)
    ]).content
    
    # 解析过渡语
    transition_dict = {}
    for line in transitions.split('\n'):
        if '->' in line and ':' in line:
            key = line.split(':')[0].strip()
            value = line.split(':')[1].strip()
            transition_dict[key] = value
    
    # 组装最终报告
    output_format = state.get('output_format', 'markdown')
    
    if output_format == 'markdown':
        full_report = f"# {state['topic']}\n\n"
        full_report += f"*生成时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"
        full_report += "## 目录\n\n"
        
        # 生成目录
        for i, draft in enumerate(sorted_drafts):
            full_report += f"{i+1}. [{draft.title}](#{draft.title.replace('','-').lower()})\n"
        
        full_report += "\n---\n\n"
        
        # 组装章节内容
        for i, draft in enumerate(sorted_drafts):
            full_report += f"## {draft.title}\n\n"
            full_report += draft.content
            
            # 添加过渡语
            if i < len(sorted_drafts) - 1:
                next_title = sorted_drafts[i+1].title
                transition_key = f"{draft.title} -> {next_title}"
                if transition_key in transition_dict:
                    full_report += f"\n\n*{transition_dict[transition_key]}*\n\n"
                else:
                    full_report += "\n\n"
            
            if i < len(sorted_drafts) - 1:
                full_report += "---\n\n"
    
    # 保存报告
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    report_filename = f"reports/report_{timestamp}.md"
    with open(report_filename, "w", encoding="utf-8") as f:
        f.write(full_report)
    
    # 更新TODO
    completed = state['completed_tasks'] + [f"生成章节: {draft.title}" for draft in sorted_drafts] + ["合并所有章节"]
    remaining = [task for task in state['todo_list'] if task not in completed]
    update_todo_list(remaining, completed)
    
    return {"final_report": full_report}

# ---------- 11. 验证节点 ----------
validator = llm.with_structured_output(ValidationResult)

def validation_node(state: State):
    print("🔍 开始验证最终报告...")
    
    validation_prompt = f"""
    请对以下报告进行全面验证：
    
    主题: {state['topic']}
    
    报告内容:
    {state['final_report'][:2000]}...  # 截取前2000字进行验证
    
    请验证以下方面并以JSON格式返回：
    1. 内容完整性和逻辑性
    2. 章节间的连贯性
    3. 事实准确性
    4. 语言质量
    5. 结构合理性
    
    返回格式：
    {{
        "is_valid": true/false,
        "issues": ["问题1", "问题2"],
        "suggestions": ["建议1", "建议2"]
    }}
    """
    
    validation = validator.invoke([
        SystemMessage(content="你是专业的内容质量验证专家，请按JSON格式返回验证结果。"),
        HumanMessage(content=validation_prompt)
    ])
    
    print(f"📊 验证结果: {'通过' if validation.is_valid else '需要改进'}")
    if not validation.is_valid:
        print("❌ 发现问题:")
        for issue in validation.issues:
            print(f"  - {issue}")
        print("💡 改进建议:")
        for suggestion in validation.suggestions:
            print(f"  - {suggestion}")
    
    # 更新TODO
    completed = state['completed_tasks'] + ["验证最终报告"]
    update_todo_list([], completed)
    
    return {"validation_result": validation}

# ---------- 12. 路由函数 ----------
def should_regenerate(state: State):
    """决定是否需要重新生成"""
    if state.get('validation_result') and not state['validation_result'].is_valid:
        return "regenerate"
    return "complete"

# ---------- 13. 构建工作流 ----------
def build_workflow():
    setup_directories()
    
    workflow = StateGraph(State)
    
    # 添加节点
    workflow.add_node("plan", plan_node)
    workflow.add_node("worker", worker_node)
    workflow.add_node("merge", merge_and_format)
    workflow.add_node("validate", validation_node)
    workflow.add_node("complete", lambda state: {"final_report": state["final_report"]})
    
    # 添加边
    workflow.add_edge(START, "plan")
    workflow.add_conditional_edges("plan", dispatch, ["worker"])
    workflow.add_edge("worker", "merge")
    workflow.add_edge("merge", "validate")
    workflow.add_conditional_edges("validate", should_regenerate, {
        "complete": "complete",
        "regenerate": "plan"  # 如果验证失败，重新开始
    })
    workflow.add_edge("complete", END)
    
    return workflow.compile()

# ---------- 14. 主程序 ----------
if __name__ == "__main__":
    print("🚀 启动增强版报告生成系统...")
    
    app = build_workflow()
    
    # 配置参数
    config = {
        "topic": "生成式 AI 如何重塑教育",
        "output_format": "markdown",
        "todo_list": [],
        "completed_tasks": [],
        "search_results": [],
        "drafts": []
    }
    
    try:
        result = app.invoke(config)
        
        print("\n" + "="*50)
        print("📊 最终报告生成完成!")
        print("="*50)
        
        # 显示文件位置
        print(f"📁 报告文件: reports/")
        print(f"📁 搜索结果: search_results/")
        print(f"📁 章节草稿: drafts/")
        print(f"📁 任务列表: todo.txt")
        
        if result.get('validation_result'):
            validation = result['validation_result']
            print(f"\n✅ 验证状态: {'通过' if validation.is_valid else '需要改进'}")
            
        print(f"\n📝 报告预览 (前500字):")
        print("-" * 30)
        print(result["final_report"][:500] + "...")
        
    except Exception as e:
        print(f"❌ 生成过程出错: {e}")
        import traceback
        traceback.print_exc()

🚀 启动增强版报告生成系统...
📋 开始制定计划...
📋 开始并行处理 10 个章节
🔄 处理章节 1: 引言：生成式 AI 的兴起
🔄 处理章节 2: 教育领域的传统挑战
🔄 处理章节 3: 生成式 AI 在教学中的应用
🔄 处理章节 5: 教师角色的转变与支持
🔄 处理章节 4: 个性化学习的实现
🔄 处理章节 6: 评估与反馈机制革新
🔄 处理章节 7: 教育资源公平性的影响
🔄 处理章节 8: 伦理与隐私问题探讨
🔄 处理章节 9: 未来教育模式的展望
🔄 处理章节 10: 结论与行动建议
🔍 搜索查询优化: 引言：生成式 AI 的兴起 介绍生成式 AI 技术的发展背景及其潜力。 -> "生成式 AI 教育应用" AND ("技术发展" OR "潜力") AND (2023 OR 2024) AND (权威研究 OR 学术报告)
🔍 搜索查询优化: 教育资源公平性的影响 研究生成式 AI 在缩小教育鸿沟方面的潜在价值。 -> "生成式 AI" AND ("教育资源公平" OR "教育鸿沟") AND (impact OR effect) AND 2023..2024 AND (review OR study)
🔍 搜索查询优化: 评估与反馈机制革新 分析 AI 在作业批改、学习效果评估中的作用。 -> "生成式AI 作业批改 学习评估" AND ("教育技术" OR "智能教学") AND (2020..2024) site:edu OR site:gov
🔍 搜索查询优化: 教师角色的转变与支持 讨论 AI 技术如何改变教师职责并为其提供新工具。 -> "生成式 AI 教育应用" AND "教师角色转变" AND "AI教学工具" AND (2020..2024) AND (权威 OR 研究)
🔍 搜索查询优化: 结论与行动建议 总结影响，并提出政策制定者和教育者的应对策略。 -> "生成式 AI 教育影响" AND ("政策建议" OR "教学策略") AND 2023..2024 AND (政府报告 OR 教育研究)
🔍 搜索查询优化: 个性化学习的实现 展示生成式 AI 如何为不同学生提供定制化学习体验。 -> "生成式AI" AND "个性化学习" AND ("教育应用" OR "教学创新") AND (2020..2024) AND (权威性

/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://er

✅ 完成章节 2: 教育领域的传统挑战
✅ 完成章节 3: 生成式 AI 在教学中的应用
✅ 完成章节 6: 评估与反馈机制革新
✅ 完成章节 4: 个性化学习的实现
✅ 完成章节 10: 结论与行动建议
✅ 完成章节 1: 引言：生成式 AI 的兴起
✅ 完成章节 7: 教育资源公平性的影响
✅ 完成章节 5: 教师角色的转变与支持
✅ 完成章节 9: 未来教育模式的展望
✅ 完成章节 8: 伦理与隐私问题探讨
📝 合并 10 个章节
🔍 开始验证最终报告...
📊 验证结果: 需要改进
❌ 发现问题:
  - 内容完整性部分缺失：引言部分改写内容仅覆盖了部分内容，未见后续章节如‘个性化学习的实现’、‘教师角色的转变’等的实际撰写内容。
  - 逻辑性不足：由于仅提供引言部分详细内容，其他章节标题存在但无正文，无法判断整体报告逻辑是否自洽。
  - 事实准确性存疑：部分数据来源标注模糊，如‘中国人工智能产业发展联盟’2028年预测值缺乏具体引用方式或原始链接支持；GPT-4o发布时间为2024年属于未来事件（当前时间为2025年7月）。
  - 语言质量不一致：说明部分提到‘优化逻辑衔接：每段...’但后文被截断，影响理解优化程度。
  - 结构合理性问题：目录与实际内容严重不匹配，仅提供引言内容，其余章节为空，结构失衡。
💡 改进建议:
  - 补充完整各章节正文内容，确保与目录对应，保障报告的整体性和系统性。
  - 核实未来时间点的技术发布时间和数据预测来源，确保所有引用可查证并具有权威出处。
  - 统一语言风格，在技术描述与政策分析之间保持专业性与通俗性的平衡。
  - 增强章节间的过渡语句，提升段落之间的逻辑连贯性与阅读流畅度。
  - 建议在图表或数据引用处添加脚注或尾注，便于读者追溯原始资料。
📋 开始制定计划...
📋 开始并行处理 10 个章节
🔄 处理章节 1: 引言：AI 技术的崛起
🔄 处理章节 3: 生成式 AI 在教学中的应用
🔄 处理章节 2: 教育现状与挑战
🔄 处理章节 4: 个性化学习体验
🔄 处理章节 5: 学生创新能力培养
🔄 处理章节 7: 数据隐私与伦理问题
🔄 处理章节 8: 教师角色的转变
🔄 处理章节 9: 未来教育模式展望
🔄 处理章节 10: 结论与建议
🔄 处理章节 6: 教育资源公平分配
🔍 搜索查询优化

/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)


🔍 搜索查询优化: 引言：AI 技术的崛起 介绍生成式 AI 的发展及其技术潜力。 -> "生成式 AI" AND ("教育应用" OR "教学创新") AND ("技术发展" OR "潜力") AND ("2023" OR "2024") AND (site:edu OR site:gov)


...

/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://er

✅ 完成章节 6: 学生创造力与 AI 的结合
✅ 完成章节 8: 教育资源的公平分配问题
✅ 完成章节 3: 教育领域的传统模式分析
✅ 完成章节 2: 生成式 AI 的基本概念
✅ 完成章节 4: 生成式 AI 在教学中的应用
✅ 完成章节 10: 未来教育生态系统的变化
✅ 完成章节 7: 评估与反馈机制的革新
✅ 完成章节 13: 技术局限性与潜在风险
✅ 完成章节 14: 全球范围内的实践案例分享
✅ 完成章节 9: 伦理与隐私保护的挑战
✅ 完成章节 5: 个性化学习的实现路径
✅ 完成章节 11: 政策制定与监管框架需求
✅ 完成章节 1: 引言：AI 技术的快速发展
✅ 完成章节 15: 结语：迈向 AI 赋能的教育时代
✅ 完成章节 12: 教师角色的转型与适应
📝 合并 55 个章节
🔍 开始验证最终报告...
📊 验证结果: 需要改进
❌ 发现问题:
  - 内容完整性不足：目录重复项过多，例如‘引言’、‘生成式 AI 在教学中的应用’、‘个性化学习的实现’、‘教师角色的转变’、‘评估与反馈机制’、‘伦理与隐私问题’等多次重复出现，缺乏实质性内容支撑。
  - 逻辑性混乱：多个章节标题重复且未见具体展开内容，导致整体结构逻辑不清晰，读者难以理解报告的核心论点和推导过程。
  - 章节间连贯性差：从目录来看，章节命名相似但无明确递进或区分，未能体现各部分之间的逻辑衔接关系。
  - 事实准确性无法判断：由于仅提供了目录而无正文内容，无法验证所涉及观点或数据的真实性。
  - 语言质量较低：章节标题中存在格式错误（如空格插入）、用词重复、语义模糊等问题，影响阅读体验。
  - 结构不合理：目录层级混乱，缺少必要的章节分类与组织结构，如‘基本概念’、‘应用场景’、‘挑战与问题’、‘未来展望’等模块未形成清晰体系。
💡 改进建议:
  - 精简目录结构，去除重复章节并明确每个章节的独特主题和内容定位。
  - 补充完整正文内容，确保每一章节有实质性论述以支撑整体论点。
  - 优化章节命名，使用更具描述性和差异性的标题以增强可读性和逻辑性。
  - 建立清晰的内容框架，如分为背景介绍、技术基础、教育应用、挑战分析、伦理考量、未来展望等模块。
  - 校对语言表达，修正格式错误，提升标题的准确性和专业性。
  - 增加过渡段落或小结部分，增强章节间的连贯

/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(search_result.dict(), f, ensure_ascii=False, indent=2)
/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py:154: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://er

✅ 完成章节 9: 伦理与隐私问题
✅ 完成章节 6: 教育资源的普及与公平
✅ 完成章节 2: 生成式 AI 的工作原理
✅ 完成章节 3: 传统教育模式的挑战
✅ 完成章节 7: 评估与反馈机制的优化
✅ 完成章节 8: 教师角色的转变与支持
✅ 完成章节 1: 引言：生成式 AI 的兴起
✅ 完成章节 10: 未来教育生态的展望
✅ 完成章节 4: 生成式 AI 在教学中的应用
✅ 完成章节 5: 学生学习体验的变革
📝 合并 65 个章节
🔍 开始验证最终报告...
📊 验证结果: 需要改进
❌ 发现问题:
  - 目录中存在大量重复的章节标题，如多个‘引言：生成式 AI 的兴起’、‘生成式 AI 在教学中的应用’、‘评估与反馈机制优化’等，导致内容结构混乱。
  - 章节标题缺乏明确的逻辑递进关系，整体结构不清晰，无法体现报告的分析框架。
  - 部分章节标题过于相似（如‘教育领域的传统挑战’、‘教育现状与挑战’、‘教育的现状与挑战’），难以区分其具体内容差异，影响可读性。
  - 部分内容截断（如第47条未完整显示），可能影响信息完整性。
  - 语言表达不够专业，部分标题用词重复、冗余，缺乏学术报告应有的严谨性和规范性。
💡 改进建议:
  - 合并重复章节，并重新命名以反映不同角度或深度的内容，确保每个章节具有唯一性和独立性。
  - 制定清晰的逻辑结构，例如按照‘背景介绍 → 问题分析 → 技术原理 → 应用场景 → 影响与挑战 → 未来展望’的顺序组织内容。
  - 统一术语和表述方式，避免如‘教育现状与挑战’与‘教育的现状与挑战’这样仅微小差别的标题。
  - 补充完整缺失的部分（如第47节），并确保整篇报告内容完整无误。
  - 提升语言质量，使用更专业的术语和更规范的表达方式，增强报告的学术性和可读性。
📋 开始制定计划...
📋 开始并行处理 12 个章节
❌ 生成过程出错: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langc

Traceback (most recent call last):
  File "/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/3768850566.py", line 487, in <module>
    result = app.invoke(config)
             ^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/lib/python3.12/site-packages/langgraph/pregel/__init__.py", line 2844, in invoke
    for chunk in self.stream(
  File "/opt/miniconda3/lib/python3.12/site-packages/langgraph/pregel/__init__.py", line 2559, in stream
    raise GraphRecursionError(msg)
langgraph.errors.GraphRecursionError: Recursion limit of 25 reached without hitting a stop condition. You can increase the limit by setting the `recursion_limit` config key.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/GRAPH_RECURSION_LIMIT


In [ ]:
import re
import requests
from typing import List, Dict, Optional
from pydantic import BaseModel, Field

class ImageInfo(BaseModel):
    url: str
    description: str
    source: str
    relevance_score: float = 0.0
    query_used: str = ""

def extract_keywords_from_content(search_results: List[Dict], topic: str) -> List[str]:
    """从搜索结果中提取关键词用于图片搜索"""
    
    # 合并所有搜索结果内容
    all_content = ""
    for result in search_results:
        content = result.get("content", "")
        all_content += content + " "
    
    # 使用LLM提取关键词
    keyword_prompt = f"""
    主题: {topic}
    搜索结果内容: {all_content[:1000]}...
    
    请从这些内容中提取3-5个最适合用于图片搜索的关键词或短语。
    
    要求：
    1. 选择具有强烈视觉化特征的词汇
    2. 避免过于抽象的概念
    3. 每个关键词不超过3个单词
    4. 按照相关性排序
    5. 用英文返回（便于图片搜索）
    
    返回格式：
    keyword1, keyword2, keyword3, keyword4, keyword5
    
    只返回关键词，用逗号分隔。
    """
    
    try:
        # 这里使用您的LLM实例
        keywords_text = llm.invoke([
            SystemMessage(content="你是关键词提取专家，专门为图片搜索提取最佳关键词。"),
            HumanMessage(content=keyword_prompt)
        ]).content.strip()
        
        # 解析关键词
        keywords = [kw.strip() for kw in keywords_text.split(',') if kw.strip()]
        return keywords[:5]  # 最多5个关键词
    
    except Exception as e:
        print(f"提取关键词时出错: {e}")
        # 回退到简单的关键词提取
        return [topic.split()[:2]]  # 使用主题的前两个词作为关键词

def search_pexels_images(keywords: List[str], max_images: int = 3) -> List[ImageInfo]:
    """使用Pexels API搜索图片"""
    
    # Pexels API密钥 - 需要在 https://www.pexels.com/api/ 申请
    PEXELS_API_KEY = "You API Key"  # 替换为您的API密钥
    
    if not PEXELS_API_KEY or PEXELS_API_KEY == "YOUR_PEXELS_API_KEY":
        print("⚠️ 未设置Pexels API密钥，跳过Pexels搜索")
        return []
    
    images = []
    
    for keyword in keywords:
        if len(images) >= max_images:
            break
            
        try:
            # Pexels API请求
            url = f"https://api.pexels.com/v1/search"
            headers = {
                "Authorization": PEXELS_API_KEY
            }
            params = {
                "query": keyword,
                "per_page": 2,  # 每个关键词最多2张图片
                "size": "medium"
            }
            
            response = requests.get(url, headers=headers, params=params, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                photos = data.get("photos", [])
                
                for photo in photos:
                    if len(images) >= max_images:
                        break
                    
                    images.append(ImageInfo(
                        url=photo["src"]["medium"],
                        description=f"关于 '{keyword}' 的图片",
                        source="Pexels",
                        relevance_score=0.7,
                        query_used=keyword
                    ))
                    
                print(f"✅ 从Pexels找到 {len(photos)} 张关于 '{keyword}' 的图片")
            
            else:
                print(f"❌ Pexels API请求失败: {response.status_code}")
                
        except Exception as e:
            print(f"Pexels搜索出错 '{keyword}': {e}")
            continue
    
    return images

def search_images_with_browser_use(keywords: List[str], max_images: int = 3) -> List[ImageInfo]:
    """使用Browser Use搜索图片（需要browser_use库）"""
    
    try:
        # 这里假设您有browser_use的实现
        # 如果没有，可以跳过这个函数或使用其他方式
        
        # 示例实现（需要根据实际的browser_use库调整）
        # from browser_use import BrowserUse
        
        images = []
        
        for keyword in keywords:
            if len(images) >= max_images:
                break
                
            try:
                # 使用browser搜索图片
                search_query = f"{keyword} high quality images"
                
                # 这里需要实际的browser_use实现
                # browser = BrowserUse()
                # results = browser.search_images(search_query, limit=2)
                
                # 暂时返回占位符结果
                # 实际实现时需要解析browser返回的结果
                placeholder_results = [
                    {
                        "url": f"https://via.placeholder.com/800x600?text={keyword.replace(' ', '+')}",
                        "title": f"Image about {keyword}",
                        "source": "Browser Search"
                    }
                ]
                
                for result in placeholder_results:
                    if len(images) >= max_images:
                        break
                    
                    images.append(ImageInfo(
                        url=result["url"],
                        description=f"Browser搜索: {keyword}",
                        source="Browser Use",
                        relevance_score=0.6,
                        query_used=keyword
                    ))
                
                print(f"✅ 通过Browser Use找到 {len(placeholder_results)} 张关于 '{keyword}' 的图片")
                
            except Exception as e:
                print(f"Browser Use搜索出错 '{keyword}': {e}")
                continue
                
    except ImportError:
        print("⚠️ Browser Use库未安装，跳过Browser Use搜索")
        return []
    
    except Exception as e:
        print(f"Browser Use搜索出错: {e}")
        return []
    
    return images


def extract_images_from_search(search_results: List[Dict], topic: str = "", max_images: int = 3) -> List[ImageInfo]:
    """从搜索结果中提取图片信息，如果没有找到则通过关键词搜索"""
    
    images = []
    
    # 第一步：从搜索结果中直接提取图片URL
    for result in search_results:
        content = result.get("content", "")
        url = result.get("url", "")
        
        # 使用正则表达式查找图片URL
        image_patterns = [
            r'https?://[^\s]+\.(?:jpg|jpeg|png|gif|webp|svg)',
            r'!\[.*?\]\((https?://[^\s]+\.(?:jpg|jpeg|png|gif|webp|svg))\)',
            r'<img[^>]+src="([^"]+)"'
        ]
        
        for pattern in image_patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    img_url = match[0]
                else:
                    img_url = match
                
                images.append(ImageInfo(
                    url=img_url,
                    description=f"来自 {url}",
                    source=url,
                    relevance_score=0.8,
                    query_used="直接提取"
                ))
    
    # 去重
    unique_images = {}
    for img in images:
        if img.url not in unique_images:
            unique_images[img.url] = img
    
    found_images = list(unique_images.values())[:max_images]
    
    print(f"📸 直接找到 {len(found_images)} 张图片")
    
    # 第二步：如果图片数量不足，通过关键词搜索
    if len(found_images) < max_images and search_results:
        remaining_slots = max_images - len(found_images)
        
        print(f"🔍 图片数量不足，开始关键词搜索 (需要 {remaining_slots} 张)")
        
        # 提取关键词
        keywords = extract_keywords_from_content(search_results, topic)
        
        if keywords:
            print(f"🔑 提取的关键词: {', '.join(keywords)}")
            
            # 尝试多个图片源
            additional_images = []
            
            # 1. 尝试Pexels
            try:
                pexels_images = search_pexels_images(keywords, remaining_slots)
                additional_images.extend(pexels_images)
                print(f"📸 Pexels找到 {len(pexels_images)} 张图片")
            except Exception as e:
                print(f"❌ Pexels搜索失败: {e}")
            
            
            # 3. 如果还不够，尝试Browser Use
            if len(additional_images) < remaining_slots:
                try:
                    browser_images = search_images_with_browser_use(keywords, remaining_slots - len(additional_images))
                    additional_images.extend(browser_images)
                    print(f"📸 Browser Use找到 {len(browser_images)} 张图片")
                except Exception as e:
                    print(f"❌ Browser Use搜索失败: {e}")
            
            # 合并结果
            found_images.extend(additional_images[:remaining_slots])
    
    final_images = found_images[:max_images]
    
    print(f"✅ 总共找到 {len(final_images)} 张图片")
    
    # 按相关性排序
    final_images.sort(key=lambda x: x.relevance_score, reverse=True)
    
    return final_images

In [ ]:
import operator
import os
import json
import datetime
import hashlib
from typing import Annotated, List, TypedDict, Dict, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.messages import SystemMessage, HumanMessage

# ---------- 1. 全局配置 ----------
#llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
search = TavilySearchResults(max_results=3)
MAX_REGEN = 3

# ---------- 2. 版本控制数据结构 ----------
class SectionVersion(BaseModel):
    version: int
    title: str
    content: str
    content_hash: str
    timestamp: str
    is_current: bool = False

class SectionHistory(BaseModel):
    section_index: int
    base_title: str
    versions: List[SectionVersion] = []
    current_version: int = 0
    
    def add_version(self, title: str, content: str) -> bool:
        """添加新版本，检查是否重复"""
        content_hash = hashlib.md5(content.encode()).hexdigest()
        
        # 检查是否与现有版本重复
        for version in self.versions:
            if version.content_hash == content_hash:
                print(f"⚠️ 跳过重复内容: {title}")
                return False
        
        # 将之前的版本标记为非当前
        for version in self.versions:
            version.is_current = False
        
        # 添加新版本
        new_version = SectionVersion(
            version=len(self.versions) + 1,
            title=title,
            content=content,
            content_hash=content_hash,
            timestamp=datetime.datetime.now().isoformat(),
            is_current=True
        )
        
        self.versions.append(new_version)
        self.current_version = new_version.version
        
        print(f"✅ 新版本: {title} (v{new_version.version})")
        return True
    
    def get_current(self) -> Optional[SectionVersion]:
        """获取当前版本"""
        for version in self.versions:
            if version.is_current:
                return version
        return None

class ReportVersion(BaseModel):
    version: int
    timestamp: str
    section_versions: Dict[int, int]
    validation_passed: bool = False
    regenerate_reason: str = ""

# ---------- 3. 修正的合并函数 ----------
def merge_section_histories(
    left: List[SectionHistory], 
    right: List[SectionHistory]
) -> List[SectionHistory]:
    """
    合并两个章节历史列表
    LangGraph 要求签名为 (a, b) -> c
    """
    # 合并所有历史记录
    all_histories = left + right
    
    # 按 section_index 分组，保留最新的版本
    merged = {}
    for history in all_histories:
        key = history.section_index
        if key not in merged:
            merged[key] = history
        else:
            # 比较版本，保留更新的
            if history.current_version > merged[key].current_version:
                merged[key] = history
    
    # 按索引排序返回
    return sorted(merged.values(), key=lambda h: h.section_index)

# ---------- 4. 修正后的状态结构 ----------
class State(TypedDict):
    topic: str
    plan: List[Dict]
    # 修复：使用正确签名的合并函数
    section_histories: Annotated[List[SectionHistory], merge_section_histories]
    report_versions: List[ReportVersion]
    current_report_version: int
    regenerate_count: int
    search_results: Annotated[List[Dict], operator.add]
    final_report: str
    validation_result: Optional[Dict]
    output_format: str
    todo_list: List[str]
    completed_tasks: List[str]

class WorkerState(TypedDict):
    section_info: Dict
    section_index: int
    topic: str
    regenerate_count: int
    validation_issues: List[str]
    existing_histories: List[SectionHistory]

# ---------- 5. 工具函数 ----------
def setup_directories():
    """创建工作目录"""
    dirs = ["versions", "search_results", "reports", "drafts"]
    for dir_name in dirs:
        os.makedirs(dir_name, exist_ok=True)

def save_version_history(section_histories: List[SectionHistory], version: int):
    """保存版本历史到文件"""
    version_dir = f"versions/v{version}"
    os.makedirs(version_dir, exist_ok=True)
    
    for history in section_histories:
        filename = f"{version_dir}/section_{history.section_index:02d}_history.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(history.dict(), f, ensure_ascii=False, indent=2)

def load_version_history(version: int) -> List[SectionHistory]:
    """从文件加载版本历史"""
    version_dir = f"versions/v{version}"
    histories = []
    
    if os.path.exists(version_dir):
        files = [f for f in os.listdir(version_dir) if f.endswith('_history.json')]
        for filename in sorted(files):
            filepath = os.path.join(version_dir, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
                histories.append(SectionHistory(**data))
    
    return histories

def update_todo_list(todo_list: List[str], completed_tasks: List[str], version: int):
    """更新TODO列表"""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(f"todo_v{version}.txt", "w", encoding="utf-8") as f:
        f.write(f"=== 报告生成任务列表 v{version} ({timestamp}) ===\n\n")
        
        f.write("## 已完成任务:\n")
        for task in completed_tasks:
            f.write(f"✅ {task}\n")
        
        f.write("\n## 待完成任务:\n")
        for task in todo_list:
            f.write(f"🔘 {task}\n")
        
        f.write(f"\n## 进度: {len(completed_tasks)}/{len(completed_tasks) + len(todo_list)}\n")

def optimize_search_query(section_info: Dict, topic: str, history: SectionHistory = None) -> str:
    """优化搜索查询"""
    context = ""
    if history and history.versions:
        recent_titles = [v.title for v in history.versions[-2:]]
        context = f"之前版本标题: {', '.join(recent_titles)}"
    
    optimization_prompt = f"""
    主题: {topic}
    章节标题: {section_info['title']}
    章节描述: {section_info['description']}
    {context}
    
    请生成一个优化的搜索查询，要求：
    1. 包含核心关键词
    2. 避免重复之前的内容
    3. 寻找最新信息和案例
    4. 限制50字以内
    
    直接返回搜索查询。
    """
    
    optimized_query = llm.invoke([
        SystemMessage(content="你是搜索优化专家，专门基于历史版本优化搜索策略。"),
        HumanMessage(content=optimization_prompt)
    ]).content.strip()
    
    return optimized_query

def process_search_results(search_results: List[Dict], section_info: Dict) -> str:
    """处理搜索结果"""
    if not search_results:
        return "无搜索结果"
    
    content = "\n".join([doc.get("content", "")[:300] for doc in search_results])
    
    process_prompt = f"""
    章节: {section_info['title']}
    搜索结果: {content}
    
    请整合关键信息，去除重复内容，提取核心要点。
    """
    
    processed = llm.invoke([
        SystemMessage(content="你是信息整合专家。"),
        HumanMessage(content=process_prompt)
    ]).content
    
    return processed

# ---------- 6. 计划节点 ----------
def plan_node(state: State):
    regenerate_count = state.get('regenerate_count', 0)
    
    if regenerate_count > 0:
        print(f"🔄 重新生成 (第 {regenerate_count} 次)")
        # 重新生成时加载现有历史
        current_version = state.get('current_report_version', 1)
        existing_histories = load_version_history(current_version - 1) if current_version > 1 else []
        
        return {
            "regenerate_count": regenerate_count,
            "current_report_version": current_version,
            "section_histories": existing_histories
        }
    
    print("📋 初始计划制定...")
    
    plan_prompt = f"""为"{state['topic']}"写一份详细的大纲，每节含标题和简短描述。
    
    请以 JSON 格式返回：
    {{
        "sections": [
            {{"title": "章节标题", "description": "章节描述"}}
        ]
    }}
    """
    
    class PlanSchema(BaseModel):
        sections: List[Dict] = Field(description="章节列表")
    
    planner = llm.with_structured_output(PlanSchema)
    plan = planner.invoke([
        SystemMessage(content="你是专业的报告规划师，请按照JSON格式返回报告大纲。"),
        HumanMessage(content=plan_prompt)
    ])
    
    # 初始化章节历史
    section_histories = []
    for i, section in enumerate(plan.sections):
        history = SectionHistory(
            section_index=i,
            base_title=section['title']
        )
        section_histories.append(history)
    
    # 创建首个报告版本记录
    report_version = ReportVersion(
        version=1,
        timestamp=datetime.datetime.now().isoformat(),
        section_versions={},
        regenerate_reason="初始生成"
    )
    
    todo_list = [f"生成章节: {section['title']}" for section in plan.sections]
    todo_list.extend(["合并章节", "验证报告"])
    
    update_todo_list(todo_list, ["制定计划"], 1)
    
    return {
        "plan": plan.sections,
        "section_histories": section_histories,
        "report_versions": [report_version],
        "current_report_version": 1,
        "regenerate_count": 0,
        "todo_list": todo_list,
        "completed_tasks": ["制定计划"]
    }

# ---------- 7. Worker节点 ----------
def worker_node(state: WorkerState):
    section_info = state['section_info']
    index = state['section_index']
    topic = state['topic']
    regenerate_count = state.get('regenerate_count', 0)
    validation_issues = state.get('validation_issues', [])
    existing_histories = state.get('existing_histories', [])
    
    print(f"🔄 处理章节 {index + 1}: {section_info['title']}")
    
    # 获取当前章节历史
    current_history = None
    for history in existing_histories:
        if history.section_index == index:
            current_history = history
            break
    
    if not current_history:
        current_history = SectionHistory(
            section_index=index,
            base_title=section_info['title']
        )
    
    # 检查是否需要重新生成
    current_version = current_history.get_current()
    section_issues = [issue for issue in validation_issues if f"章节{index + 1}" in issue]
    
    # 如果是首次生成或有问题需要改进
    if not current_version or section_issues or regenerate_count > 0:
        
        # 优化搜索
        optimized_query = optimize_search_query(section_info, topic, current_history)
        
        # 执行搜索
        try:
            search_results = search.invoke(optimized_query)
            processed_content = process_search_results(search_results, section_info)
        except Exception as e:
            print(f"搜索错误: {e}")
            processed_content = "无法获取相关信息"
            search_results = []
        
        # 构建写作提示
        improvement_context = ""
        if current_version:
            improvement_context = f"""
            当前版本 (v{current_version.version}):
            标题: {current_version.title}
            内容长度: {len(current_version.content)} 字
            
            需要改进的问题:
            {', '.join(section_issues)}
            
            请在现有基础上改进，不要简单重复。
            """
        
        writer_prompt = f"""
        你是专业的报告写作专家。
        
        章节信息:
        - 标题: {section_info['title']}
        - 描述: {section_info['description']}
        - 版本: {current_version.version + 1 if current_version else 1}
        
        {improvement_context}
        
        搜索素材:
        {processed_content}
        
        请{"改进现有内容" if current_version else "创作新内容"}，要求：
        1. 避免与历史版本重复
        2. 针对具体问题进行改进
        3. 使用最新信息和案例
        4. 保持专业性和可读性
        5. 长度400-600字
        
        直接返回改进后的内容，使用Markdown格式。
        """
        
        # 生成新版本
        new_content = llm.invoke([
            SystemMessage(content="你是专业的报告写作专家，专注于基于反馈的迭代改进。"),
            HumanMessage(content=writer_prompt)
        ]).content
        
        # 添加到历史记录
        if current_history.add_version(section_info['title'], new_content):
            # 保存当前版本到文件
            version_num = current_history.current_version
            draft_filename = f"drafts/section_{index:02d}_v{version_num}_{section_info['title'].replace(' ', '_')}.md"
            with open(draft_filename, "w", encoding="utf-8") as f:
                f.write(f"# {section_info['title']} (v{version_num})\n\n{new_content}")
            
            print(f"✅ 章节 {index + 1} 已更新到 v{version_num}")
    
    else:
        print(f"✅ 章节 {index + 1} 无需更新")
    
    # 返回单个历史记录
    return {
        "section_histories": [current_history],
        "search_results": [{
            "section_index": index,
            "query": optimized_query if 'optimized_query' in locals() else "",
            "results": search_results if 'search_results' in locals() else []
        }]
    }

# ---------- 8. 分发节点 ----------
def dispatch(state: State):
    print(f"📋 分发任务到 {len(state['plan'])} 个worker")
    
    # 获取验证问题
    validation_issues = []
    if state.get('validation_result'):
        validation_issues = state['validation_result'].get('issues', [])
    
    existing_histories = state.get('section_histories', [])
    
    return [
        Send("worker", {
            "section_info": section,
            "section_index": i,
            "topic": state['topic'],
            "regenerate_count": state.get('regenerate_count', 0),
            "validation_issues": validation_issues,
            "existing_histories": existing_histories
        })
        for i, section in enumerate(state["plan"])
    ]

# ---------- 9. 合并节点 ----------
def merge_node(state: State):
    current_version = state.get('current_report_version', 1)
    regenerate_count = state.get('regenerate_count', 0)
    
    print(f"📝 合并报告 v{current_version} (重新生成次数: {regenerate_count})")
    
    # 获取当前版本的章节内容
    current_sections = []
    section_version_map = {}
    
    # 确保按索引排序
    sorted_histories = sorted(state['section_histories'], key=lambda h: h.section_index)
    
    for history in sorted_histories:
        current_section = history.get_current()
        if current_section:
            current_sections.append({
                'index': history.section_index,
                'title': current_section.title,
                'content': current_section.content,
                'version': current_section.version
            })
            section_version_map[history.section_index] = current_section.version
    
    # 生成过渡内容
    transition_prompt = f"""
    主题: {state['topic']}
    章节标题: {[s['title'] for s in current_sections]}
    
    为相邻章节生成简洁的过渡语，每个1-2句。
    """
    
    transitions = llm.invoke([
        SystemMessage(content="你是文章结构专家。"),
        HumanMessage(content=transition_prompt)
    ]).content
    
    # 组装最终报告
    report = f"# {state['topic']}\n\n"
    report += f"*版本: v{current_version} | 生成时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"
    
    # 版本信息摘要
    report += "## 版本信息\n\n"
    for section in current_sections:
        report += f"- {section['title']}: v{section['version']}\n"
    
    if regenerate_count > 0:
        report += f"- 重新生成次数: {regenerate_count}\n"
    
    report += "\n---\n\n"
    
    # 目录
    report += "## 目录\n\n"
    for i, section in enumerate(current_sections):
        report += f"{i+1}. [{section['title']}](#{section['title'].replace(' ', '-').lower()})\n"
    report += "\n---\n\n"
    
    # 章节内容
    for i, section in enumerate(current_sections):
        report += f"## {section['title']}\n\n"
        report += section['content']
        
        if i < len(current_sections) - 1:
            report += "\n\n*[过渡到下一章节]*\n\n"
        
        report += "\n\n---\n\n"
    
    # 保存报告
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"reports/report_v{current_version}_{timestamp}.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(report)
    
    # 保存版本历史
    save_version_history(state['section_histories'], current_version)
    
    # 更新报告版本记录
    report_versions = state.get('report_versions', [])
    current_report_version = ReportVersion(
        version=current_version,
        timestamp=datetime.datetime.now().isoformat(),
        section_versions=section_version_map,
        regenerate_reason=f"第 {regenerate_count} 次重新生成" if regenerate_count > 0 else "正常生成"
    )
    
    # 更新或添加当前版本记录
    updated_versions = [v for v in report_versions if v.version != current_version]
    updated_versions.append(current_report_version)
    
    return {
        "final_report": report,
        "report_versions": updated_versions
    }

# ---------- 10. 验证节点 ----------
def validation_node(state: State):
    current_version = state.get('current_report_version', 1)
    print(f"🔍 验证报告 v{current_version}...")
    
    validation_prompt = f"""
    请验证以下报告的质量：
    
    主题: {state['topic']}
    版本: v{current_version}
    
    报告内容 (前1500字):
    {state['final_report'][:1500]}...
    
    请从以下方面评估并返回JSON格式：
    1. 内容完整性和深度
    2. 逻辑结构和连贯性
    3. 语言质量和专业性
    4. 创新性和时效性
    
    返回格式：
    {{
        "is_valid": true/false,
        "score": 8.5,
        "issues": ["具体问题1", "具体问题2"],
        "suggestions": ["改进建议1", "改进建议2"]
    }}
    """
    
    class ValidationSchema(BaseModel):
        is_valid: bool
        score: float
        issues: List[str]
        suggestions: List[str]
    
    validator = llm.with_structured_output(ValidationSchema)
    validation = validator.invoke([
        SystemMessage(content="你是专业的内容质量评估专家，请按JSON格式返回验证结果。"),
        HumanMessage(content=validation_prompt)
    ])
    
    print(f"📊 验证结果: 得分 {validation.score}/10 | {'通过' if validation.is_valid else '需要改进'}")
    
    if not validation.is_valid:
        print("⚠️ 发现问题:")
        for issue in validation.issues:
            print(f"  - {issue}")
    
    # 更新报告版本的验证状态
    report_versions = state.get('report_versions', [])
    for version in report_versions:
        if version.version == current_version:
            version.validation_passed = validation.is_valid
            break
    
    return {
        "validation_result": validation.dict(),
        "report_versions": report_versions
    }

# ---------- 11. 路由函数 ----------
def should_regenerate(state: State):
    """决定是否重新生成"""
    validation = state.get('validation_result', {})
    regenerate_count = state.get('regenerate_count', 0)
    
    if not validation.get('is_valid', True) and regenerate_count < MAX_REGEN:
        return "regenerate"
    return "complete"

def regenerate_routing(state: State):
    """重新生成时的路由"""
    regenerate_count = state.get('regenerate_count', 0) + 1
    current_version = state.get('current_report_version', 1) + 1
    
    print(f"🔄 开始第 {regenerate_count} 次重新生成 (v{current_version})")
    
    return {
        "regenerate_count": regenerate_count,
        "current_report_version": current_version
    }

# ---------- 12. 构建工作流 ----------
def build_workflow():
    setup_directories()
    
    workflow = StateGraph(State)
    
    # 添加节点
    workflow.add_node("plan", plan_node)
    workflow.add_node("worker", worker_node)
    workflow.add_node("merge", merge_node)
    workflow.add_node("validate", validation_node)
    workflow.add_node("regenerate", regenerate_routing)
    workflow.add_node("complete", lambda state: state)
    
    # 添加边
    workflow.add_edge(START, "plan")
    workflow.add_conditional_edges("plan", dispatch, ["worker"])
    workflow.add_edge("worker", "merge")
    workflow.add_edge("merge", "validate")
    workflow.add_conditional_edges("validate", should_regenerate, {
        "complete": "complete",
        "regenerate": "regenerate"
    })
    workflow.add_edge("regenerate", "worker")
    workflow.add_edge("complete", END)
    
    return workflow.compile()

# ---------- 13. 主程序 ----------
if __name__ == "__main__":
    print("🚀 启动版本控制报告生成系统...")
    
    app = build_workflow()
    
    config = {
        "topic": "生成式 AI 如何重塑教育",
        "regenerate_count": 0,
        "current_report_version": 1,
        "output_format": "markdown",
        "section_histories": [],
        "report_versions": [],
        "search_results": [],
        "todo_list": [],
        "completed_tasks": []
    }
    
    try:
        # 设置递归限制
        result = app.invoke(config, config={"recursion_limit": 50})
        
        print("\n" + "="*50)
        print("📊 报告生成完成!")
        print("="*50)
        
        # 显示版本信息
        current_version = result.get('current_report_version', 1)
        print(f"\n📋 最终版本: v{current_version}")
        
        print("\n📁 生成的文件:")
        print(f"  - 最终报告: reports/report_v{current_version}_*.md")
        print(f"  - 章节草稿: drafts/section_*_v*_*.md")
        print(f"  - 版本历史: versions/v*/")
        print(f"  - 任务列表: todo_v{current_version}.txt")
        
        validation = result.get('validation_result', {})
        if validation:
            print(f"\n✅ 最终得分: {validation.get('score', 0)}/10")
            print(f"📊 状态: {'通过' if validation.get('is_valid', False) else '需要改进'}")
        
        # 显示章节版本信息
        print(f"\n📖 章节版本:")
        for history in sorted(result.get('section_histories', []), key=lambda h: h.section_index):
            current = history.get_current()
            if current:
                print(f"  {history.section_index + 1}. {current.title} (v{current.version})")
        
        print(f"\n📝 最终报告预览:")
        print("-" * 30)
        print(result["final_report"][:500] + "...")
        
    except Exception as e:
        print(f"❌ 生成失败: {e}")
        import traceback
        traceback.print_exc()
import operator
import os
import json
import datetime
import hashlib
import re
from typing import Annotated, List, TypedDict, Dict, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from langchain_core.messages import SystemMessage, HumanMessage

# ---------- 1. 全局配置 ----------
#llm = ChatOpenAI(model="gpt-4o", temperature=0.7)
search = TavilySearchResults(max_results=5)  # 增加搜索结果数量
MAX_REGEN = 3

# ---------- 2. 增强的数据结构 ----------
class ImageInfo(BaseModel):
    url: str
    description: str
    source: str
    relevance_score: float = 0.0

class SectionVersion(BaseModel):
    version: int
    title: str
    content: str
    content_hash: str
    timestamp: str
    is_current: bool = False
    images: List[ImageInfo] = []  # 新增：关联的图片

class SectionHistory(BaseModel):
    section_index: int
    base_title: str
    versions: List[SectionVersion] = []
    current_version: int = 0
    
    def add_version(self, title: str, content: str, images: List[ImageInfo] = None) -> bool:
        """添加新版本，检查是否重复"""
        content_hash = hashlib.md5(content.encode()).hexdigest()
        
        # 检查是否与现有版本重复
        for version in self.versions:
            if version.content_hash == content_hash:
                print(f"⚠️ 跳过重复内容: {title}")
                return False
        
        # 将之前的版本标记为非当前
        for version in self.versions:
            version.is_current = False
        
        # 添加新版本
        new_version = SectionVersion(
            version=len(self.versions) + 1,
            title=title,
            content=content,
            content_hash=content_hash,
            timestamp=datetime.datetime.now().isoformat(),
            is_current=True,
            images=images or []
        )
        
        self.versions.append(new_version)
        self.current_version = new_version.version
        
        print(f"✅ 新版本: {title} (v{new_version.version}) - 包含 {len(new_version.images)} 张图片")
        return True
    
    def get_current(self) -> Optional[SectionVersion]:
        """获取当前版本"""
        for version in self.versions:
            if version.is_current:
                return version
        return None

class TransitionInfo(BaseModel):
    from_section: int
    to_section: int
    transition_text: str
    connection_type: str  # "logical", "causal", "temporal", "contrast"

class ReportVersion(BaseModel):
    version: int
    timestamp: str
    section_versions: Dict[int, int]
    validation_passed: bool = False
    regenerate_reason: str = ""
    transitions: List[TransitionInfo] = []  # 新增：过渡信息

# ---------- 3. 合并函数 ----------
def merge_section_histories(
    left: List[SectionHistory], 
    right: List[SectionHistory]
) -> List[SectionHistory]:
    """合并两个章节历史列表"""
    all_histories = left + right
    merged = {}
    for history in all_histories:
        key = history.section_index
        if key not in merged:
            merged[key] = history
        else:
            if history.current_version > merged[key].current_version:
                merged[key] = history
    return sorted(merged.values(), key=lambda h: h.section_index)

# ---------- 4. 状态结构 ----------
class State(TypedDict):
    topic: str
    plan: List[Dict]
    section_histories: Annotated[List[SectionHistory], merge_section_histories]
    report_versions: List[ReportVersion]
    current_report_version: int
    regenerate_count: int
    search_results: Annotated[List[Dict], operator.add]
    final_report: str
    validation_result: Optional[Dict]
    output_format: str
    todo_list: List[str]
    completed_tasks: List[str]

class WorkerState(TypedDict):
    section_info: Dict
    section_index: int
    topic: str
    regenerate_count: int
    validation_issues: List[str]
    existing_histories: List[SectionHistory]

# ---------- 5. 工具函数 ----------
def setup_directories():
    """创建工作目录"""
    dirs = ["versions", "search_results", "reports", "drafts", "images"]
    for dir_name in dirs:
        os.makedirs(dir_name, exist_ok=True)

def save_version_history(section_histories: List[SectionHistory], version: int):
    """保存版本历史到文件"""
    version_dir = f"versions/v{version}"
    os.makedirs(version_dir, exist_ok=True)
    
    for history in section_histories:
        filename = f"{version_dir}/section_{history.section_index:02d}_history.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(history.dict(), f, ensure_ascii=False, indent=2)

def load_version_history(version: int) -> List[SectionHistory]:
    """从文件加载版本历史"""
    version_dir = f"versions/v{version}"
    histories = []
    
    if os.path.exists(version_dir):
        files = [f for f in os.listdir(version_dir) if f.endswith('_history.json')]
        for filename in sorted(files):
            filepath = os.path.join(version_dir, filename)
            with open(filepath, "r", encoding="utf-8") as f:
                data = json.load(f)
                histories.append(SectionHistory(**data))
    
    return histories

def update_todo_list(todo_list: List[str], completed_tasks: List[str], version: int):
    """更新TODO列表"""
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    with open(f"todo_v{version}.txt", "w", encoding="utf-8") as f:
        f.write(f"=== 报告生成任务列表 v{version} ({timestamp}) ===\n\n")
        
        f.write("## 已完成任务:\n")
        for task in completed_tasks:
            f.write(f"✅ {task}\n")
        
        f.write("\n## 待完成任务:\n")
        for task in todo_list:
            f.write(f"🔘 {task}\n")
        
        f.write(f"\n## 进度: {len(completed_tasks)}/{len(completed_tasks) + len(todo_list)}\n")
'''
def extract_images_from_search(search_results: List[Dict]) -> List[ImageInfo]:
    """从搜索结果中提取图片信息"""
    images = []
    for result in search_results:
        # 检查是否包含图片URL
        content = result.get("content", "")
        url = result.get("url", "")
        
        # 使用正则表达式查找图片URL
        image_patterns = [
            r'https?://[^\s]+\.(?:jpg|jpeg|png|gif|webp|svg)',
            r'!\[.*?\]\((https?://[^\s]+\.(?:jpg|jpeg|png|gif|webp|svg))\)',
            r'<img[^>]+src="([^"]+)"'
        ]
        
        for pattern in image_patterns:
            matches = re.findall(pattern, content, re.IGNORECASE)
            for match in matches:
                if isinstance(match, tuple):
                    img_url = match[0]
                else:
                    img_url = match
                
                images.append(ImageInfo(
                    url=img_url,
                    description=f"来自 {url}",
                    source=url,
                    relevance_score=0.8
                ))
    
    # 去重并限制数量
    unique_images = {}
    for img in images:
        if img.url not in unique_images:
            unique_images[img.url] = img
    
    return list(unique_images.values())[:3]  # 最多3张图片
'''

def optimize_search_query(section_info: Dict, topic: str, history: SectionHistory = None) -> str:
    """优化搜索查询"""
    context = ""
    if history and history.versions:
        recent_titles = [v.title for v in history.versions[-2:]]
        context = f"之前版本标题: {', '.join(recent_titles)}"
    
    optimization_prompt = f"""
    主题: {topic}
    章节标题: {section_info['title']}
    章节描述: {section_info['description']}
    {context}
    
    请生成一个优化的搜索查询，要求：
    1. 包含核心关键词和相关术语
    2. 避免重复之前的内容
    3. 寻找最新信息、案例和数据
    4. 包含可能的图表和可视化内容
    5. 限制在50字以内
    
    直接返回搜索查询。
    """
    
    optimized_query = llm.invoke([
        SystemMessage(content="你是搜索优化专家，专门基于历史版本优化搜索策略。"),
        HumanMessage(content=optimization_prompt)
    ]).content.strip()
    
    return optimized_query

def process_search_results(search_results: List[Dict], section_info: Dict) -> tuple[str, List[ImageInfo]]:
    """处理搜索结果，返回文本内容和图片信息"""
    if not search_results:
        return "无搜索结果", []
    
    # 提取图片
    images = extract_images_from_search(search_results)
    
    # 处理文本内容
    content = "\n".join([doc.get("content", "")[:400] for doc in search_results])
    
    process_prompt = f"""
    章节: {section_info['title']}
    搜索结果: {content}
    
    请整合关键信息，要求：
    1. 提取核心要点和数据
    2. 去除重复内容
    3. 按逻辑重要性排序
    4. 保持事实准确性
    5. 标注适合插入图片的位置（用[图片插入点]标记）
    
    返回整合后的素材。
    """
    
    processed = llm.invoke([
        SystemMessage(content="你是信息整合专家，擅长从多个来源提取和整合关键信息。"),
        HumanMessage(content=process_prompt)
    ]).content
    
    return processed, images

def insert_images_into_content(content: str, images: List[ImageInfo]) -> str:
    """将图片插入到内容中的合适位置"""
    if not images:
        return content
    
    # 找到合适的插入点
    paragraphs = content.split('\n\n')
    result_paragraphs = []
    
    image_index = 0
    for i, paragraph in enumerate(paragraphs):
        result_paragraphs.append(paragraph)
        
        # 在每隔2-3段插入一张图片
        if image_index < len(images) and i > 0 and (i + 1) % 3 == 0:
            img = images[image_index]
            image_md = f"\n\n![{img.description}]({img.url})\n*图片来源: {img.source}*\n"
            result_paragraphs.append(image_md)
            image_index += 1
    
    return '\n\n'.join(result_paragraphs)

# ---------- 6. 计划节点 ----------
def plan_node(state: State):
    regenerate_count = state.get('regenerate_count', 0)
    
    if regenerate_count > 0:
        print(f"🔄 重新生成 (第 {regenerate_count} 次)")
        current_version = state.get('current_report_version', 1)
        existing_histories = load_version_history(current_version - 1) if current_version > 1 else []
        
        return {
            "regenerate_count": regenerate_count,
            "current_report_version": current_version,
            "section_histories": existing_histories
        }
    
    print("📋 初始计划制定...")
    
    plan_prompt = f"""为"{state['topic']}"写一份详细的报告大纲，每节含标题和简短描述。
    
    要求：
    1. 逻辑结构清晰，章节间有自然的递进关系
    2. 每个章节都有明确的主题和目标
    3. 适合插入相关图表和案例
    4. 总共4-6个章节
    
    请以 JSON 格式返回：
    {{
        "sections": [
            {{"title": "章节标题", "description": "章节描述", "connection_type": "logical"}}
        ]
    }}
    
    connection_type 可以是: "logical"(逻辑递进), "causal"(因果关系), "temporal"(时间顺序), "contrast"(对比关系)
    """
    
    class PlanSchema(BaseModel):
        sections: List[Dict] = Field(description="章节列表")
    
    planner = llm.with_structured_output(PlanSchema)
    plan = planner.invoke([
        SystemMessage(content="你是专业的报告规划师，请按照JSON格式返回逻辑清晰的报告大纲。"),
        HumanMessage(content=plan_prompt)
    ])
    
    # 初始化章节历史
    section_histories = []
    for i, section in enumerate(plan.sections):
        history = SectionHistory(
            section_index=i,
            base_title=section['title']
        )
        section_histories.append(history)
    
    report_version = ReportVersion(
        version=1,
        timestamp=datetime.datetime.now().isoformat(),
        section_versions={},
        regenerate_reason="初始生成"
    )
    
    todo_list = [f"生成章节: {section['title']}" for section in plan.sections]
    todo_list.extend(["生成章节过渡", "合并章节", "验证报告"])
    
    update_todo_list(todo_list, ["制定计划"], 1)
    
    return {
        "plan": plan.sections,
        "section_histories": section_histories,
        "report_versions": [report_version],
        "current_report_version": 1,
        "regenerate_count": 0,
        "todo_list": todo_list,
        "completed_tasks": ["制定计划"]
    }

# ---------- 7. Worker节点 ----------
def worker_node(state: WorkerState):
    section_info = state['section_info']
    index = state['section_index']
    topic = state['topic']
    regenerate_count = state.get('regenerate_count', 0)
    validation_issues = state.get('validation_issues', [])
    existing_histories = state.get('existing_histories', [])
    
    print(f"🔄 处理章节 {index + 1}: {section_info['title']}")
    
    # 获取当前章节历史
    current_history = None
    for history in existing_histories:
        if history.section_index == index:
            current_history = history
            break
    
    if not current_history:
        current_history = SectionHistory(
            section_index=index,
            base_title=section_info['title']
        )
    
    # 检查是否需要重新生成
    current_version = current_history.get_current()
    section_issues = [issue for issue in validation_issues if f"章节{index + 1}" in issue]
    
    # 如果是首次生成或有问题需要改进
    if not current_version or section_issues or regenerate_count > 0:
        
        # 优化搜索
        optimized_query = optimize_search_query(section_info, topic, current_history)
        
        # 执行搜索
        try:
            search_results = search.invoke(optimized_query)
            processed_content, images = process_search_results(search_results, section_info)
        except Exception as e:
            print(f"搜索错误: {e}")
            processed_content = "无法获取相关信息"
            images = []
            search_results = []
        
        # 构建写作提示
        improvement_context = ""
        if current_version:
            improvement_context = f"""
            当前版本 (v{current_version.version}):
            标题: {current_version.title}
            内容长度: {len(current_version.content)} 字
            
            需要改进的问题:
            {', '.join(section_issues)}
            
            请在现有基础上改进，避免简单重复。
            """
        
        writer_prompt = f"""
        你是专业的报告写作专家。
        
        章节信息:
        - 标题: {section_info['title']}
        - 描述: {section_info['description']}
        - 连接类型: {section_info.get('connection_type', 'logical')}
        - 版本: {current_version.version + 1 if current_version else 1}
        
        {improvement_context}
        
        搜索素材:
        {processed_content}
        
        可用图片: {len(images)} 张
        
        请{"改进现有内容" if current_version else "创作新内容"}，要求：
        1. 避免与历史版本重复
        2. 针对具体问题进行改进
        3. 使用最新信息、案例和数据
        4. 保持专业性和可读性
        5. 长度500-700字
        6. 为下一章节做好自然过渡准备
        7. 如果有合适的数据，建议插入图表位置
        
        请直接返回改进后的内容，使用Markdown格式。
        """
        
        # 生成新版本
        new_content = llm.invoke([
            SystemMessage(content="你是专业的报告写作专家，专注于基于反馈的迭代改进。"),
            HumanMessage(content=writer_prompt)
        ]).content
        
        # 插入图片
        final_content = insert_images_into_content(new_content, images)
        
        # 添加到历史记录
        if current_history.add_version(section_info['title'], final_content, images):
            # 保存当前版本到文件
            version_num = current_history.current_version
            draft_filename = f"drafts/section_{index:02d}_v{version_num}_{section_info['title'].replace(' ', '_')}.md"
            with open(draft_filename, "w", encoding="utf-8") as f:
                f.write(f"# {section_info['title']} (v{version_num})\n\n{final_content}")
            
            # 保存图片信息
            if images:
                image_info_file = f"images/section_{index:02d}_v{version_num}_images.json"
                with open(image_info_file, "w", encoding="utf-8") as f:
                    json.dump([img.dict() for img in images], f, ensure_ascii=False, indent=2)
            
            print(f"✅ 章节 {index + 1} 已更新到 v{version_num}")
    
    else:
        print(f"✅ 章节 {index + 1} 无需更新")
    
    return {
        "section_histories": [current_history],
        "search_results": [{
            "section_index": index,
            "query": optimized_query if 'optimized_query' in locals() else "",
            "results": search_results if 'search_results' in locals() else []
        }]
    }

# ---------- 8. 分发节点 ----------
def dispatch(state: State):
    print(f"📋 分发任务到 {len(state['plan'])} 个worker")
    
    validation_issues = []
    if state.get('validation_result'):
        validation_issues = state['validation_result'].get('issues', [])
    
    existing_histories = state.get('section_histories', [])
    
    return [
        Send("worker", {
            "section_info": section,
            "section_index": i,
            "topic": state['topic'],
            "regenerate_count": state.get('regenerate_count', 0),
            "validation_issues": validation_issues,
            "existing_histories": existing_histories
        })
        for i, section in enumerate(state["plan"])
    ]

# ---------- 9. 生成章节过渡 ----------
def generate_transitions(sections: List[Dict], section_histories: List[SectionHistory]) -> List[TransitionInfo]:
    """生成章节间的自然过渡"""
    transitions = []
    
    for i in range(len(sections) - 1):
        current_section = sections[i]
        next_section = sections[i + 1]
        
        # 获取章节内容
        current_history = None
        next_history = None
        
        for history in section_histories:
            if history.section_index == i:
                current_history = history
            elif history.section_index == i + 1:
                next_history = history
        
        current_content = ""
        next_content = ""
        
        if current_history:
            current_version = current_history.get_current()
            if current_version:
                current_content = current_version.content[-300:]  # 取最后300字
        
        if next_history:
            next_version = next_history.get_current()
            if next_version:
                next_content = next_version.content[:300]  # 取前300字
        
        # 生成过渡文本
        transition_prompt = f"""
        你是专业的文章编辑专家。
        
        当前章节: {current_section['title']}
        当前章节结尾: {current_content}
        
        下一章节: {next_section['title']}
        下一章节开头: {next_content}
        
        连接类型: {next_section.get('connection_type', 'logical')}
        
        请生成一个自然、流畅的过渡段落，要求：
        1. 总结当前章节的要点
        2. 自然引出下一章节的主题
        3. 根据连接类型使用合适的过渡词
        4. 长度2-3句话
        5. 语言专业且易懂
        
        只返回过渡文本，不要其他说明。
        """
        
        transition_text = llm.invoke([
            SystemMessage(content="你是专业的文章编辑，擅长创建自然流畅的章节过渡。"),
            HumanMessage(content=transition_prompt)
        ]).content.strip()
        
        transitions.append(TransitionInfo(
            from_section=i,
            to_section=i + 1,
            transition_text=transition_text,
            connection_type=next_section.get('connection_type', 'logical')
        ))
    
    return transitions

# ---------- 10. 合并节点 ----------
def merge_node(state: State):
    current_version = state.get('current_report_version', 1)
    regenerate_count = state.get('regenerate_count', 0)
    
    print(f"📝 合并报告 v{current_version} (重新生成次数: {regenerate_count})")
    
    # 获取当前版本的章节内容
    current_sections = []
    section_version_map = {}
    
    # 确保按索引排序
    sorted_histories = sorted(state['section_histories'], key=lambda h: h.section_index)
    
    for history in sorted_histories:
        current_section = history.get_current()
        if current_section:
            current_sections.append({
                'index': history.section_index,
                'title': current_section.title,
                'content': current_section.content,
                'version': current_section.version,
                'images': current_section.images
            })
            section_version_map[history.section_index] = current_section.version
    
    # 生成章节过渡
    transitions = generate_transitions(state['plan'], state['section_histories'])
    
    # 组装最终报告
    report = f"# {state['topic']}\n\n"
    report += f"*版本: v{current_version} | 生成时间: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n"
    
    # 版本信息摘要
    report += "## 版本信息\n\n"
    for section in current_sections:
        img_count = len(section['images'])
        report += f"- {section['title']}: v{section['version']}"
        if img_count > 0:
            report += f" (包含 {img_count} 张图片)"
        report += "\n"
    
    if regenerate_count > 0:
        report += f"- 重新生成次数: {regenerate_count}\n"
    
    report += "\n---\n\n"
    
    # 目录
    report += "## 目录\n\n"
    for i, section in enumerate(current_sections):
        report += f"{i+1}. [{section['title']}](#{section['title'].replace(' ', '-').lower()})\n"
    report += "\n---\n\n"
    
    # 章节内容，使用自然过渡
    for i, section in enumerate(current_sections):
        report += f"## {section['title']}\n\n"
        report += section['content']
        
        # 添加自然过渡
        if i < len(current_sections) - 1:
            transition = None
            for t in transitions:
                if t.from_section == i:
                    transition = t
                    break
            
            if transition:
                report += f"\n\n{transition.transition_text}\n\n"
            else:
                report += "\n\n"
        
        if i < len(current_sections) - 1:
            report += "---\n\n"
    
    # 保存报告
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"reports/report_v{current_version}_{timestamp}.md"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(report)
    
    # 保存版本历史
    save_version_history(state['section_histories'], current_version)
    
    # 更新报告版本记录
    report_versions = state.get('report_versions', [])
    current_report_version = ReportVersion(
        version=current_version,
        timestamp=datetime.datetime.now().isoformat(),
        section_versions=section_version_map,
        regenerate_reason=f"第 {regenerate_count} 次重新生成" if regenerate_count > 0 else "正常生成",
        transitions=transitions
    )
    
    # 更新或添加当前版本记录
    updated_versions = [v for v in report_versions if v.version != current_version]
    updated_versions.append(current_report_version)
    
    return {
        "final_report": report,
        "report_versions": updated_versions
    }

# ---------- 11. 验证节点 ----------
def validation_node(state: State):
    current_version = state.get('current_report_version', 1)
    print(f"🔍 验证报告 v{current_version}...")
    
    validation_prompt = f"""
    请验证以下报告的质量：
    
    主题: {state['topic']}
    版本: v{current_version}
    
    报告内容 (前2000字):
    {state['final_report'][:2000]}...
    
    请从以下方面评估并返回JSON格式：
    1. 内容完整性和深度 (0-10分)
    2. 逻辑结构和连贯性 (0-10分)
    3. 语言质量和专业性 (0-10分)
    4. 创新性和时效性 (0-10分)
    5. 图表和视觉元素 (0-10分)
    6. 章节过渡的自然性 (0-10分)
    
    验证通过条件：总分 >= 8.0，且没有严重问题
    
    返回格式：
    {{
        "is_valid": true/false,
        "score": 8.5,
        "detailed_scores": {{
            "content": 8.0,
            "logic": 9.0,
            "language": 8.5,
            "innovation": 8.0,
            "visuals": 7.5,
            "transitions": 9.0
        }},
        "issues": ["具体问题1", "具体问题2"],
        "suggestions": ["改进建议1", "改进建议2"]
    }}
    """
    
    class ValidationSchema(BaseModel):
        is_valid: bool
        score: float
        detailed_scores: Dict[str, float]
        issues: List[str]
        suggestions: List[str]
    
    validator = llm.with_structured_output(ValidationSchema)
    validation = validator.invoke([
        SystemMessage(content="你是专业的内容质量评估专家，请按JSON格式返回详细的验证结果。"),
        HumanMessage(content=validation_prompt)
    ])
    
    print(f"📊 验证结果: 总分 {validation.score}/10")
    print(f"📋 详细得分:")
    for aspect, score in validation.detailed_scores.items():
        print(f"  - {aspect}: {score}/10")
    
    print(f"✅ 状态: {'通过' if validation.is_valid else '需要改进'}")
    
    if not validation.is_valid:
        print("⚠️ 发现问题:")
        for issue in validation.issues:
            print(f"  - {issue}")
    
    # 更新报告版本的验证状态
    report_versions = state.get('report_versions', [])
    for version in report_versions:
        if version.version == current_version:
            version.validation_passed = validation.is_valid
            break
    
    return {
        "validation_result": validation.dict(),
        "report_versions": report_versions
    }

# ---------- 12. 路由函数 ----------
def should_regenerate(state: State):
    """决定是否重新生成 - 验证通过就直接结束"""
    validation = state.get('validation_result', {})
    regenerate_count = state.get('regenerate_count', 0)
    
    # 如果验证通过，直接完成
    if validation.get('is_valid', False):
        print("✅ 验证通过，报告生成完成！")
        return "complete"
    
    # 如果验证未通过且未达到最大重试次数，重新生成
    if regenerate_count < MAX_REGEN:
        print(f"⚠️ 验证未通过，准备第 {regenerate_count + 1} 次重新生成...")
        return "regenerate"
    
    # 达到最大重试次数，强制完成
    print(f"🚫 已达最大重试次数 ({MAX_REGEN})，强制完成生成。")
    return "complete"

def regenerate_routing(state: State):
    """重新生成时的路由"""
    regenerate_count = state.get('regenerate_count', 0) + 1
    current_version = state.get('current_report_version', 1) + 1
    
    validation = state.get('validation_result', {})
    
    print(f"🔄 开始第 {regenerate_count} 次重新生成 (v{current_version})")
    print(f"🎯 主要问题: {', '.join(validation.get('issues', []))}")
    
    return {
        "regenerate_count": regenerate_count,
        "current_report_version": current_version
    }

# ---------- 13. 构建工作流 ----------
def build_workflow():
    setup_directories()
    
    workflow = StateGraph(State)
    
    # 添加节点
    workflow.add_node("plan", plan_node)
    workflow.add_node("worker", worker_node)
    workflow.add_node("merge", merge_node)
    workflow.add_node("validate", validation_node)
    workflow.add_node("regenerate", regenerate_routing)
    workflow.add_node("complete", lambda state: state)
    
    # 添加边
    workflow.add_edge(START, "plan")
    workflow.add_conditional_edges("plan", dispatch, ["worker"])
    workflow.add_edge("worker", "merge")
    workflow.add_edge("merge", "validate")
    workflow.add_conditional_edges("validate", should_regenerate, {
        "complete": "complete",
        "regenerate": "regenerate"
    })
    workflow.add_edge("regenerate", "worker")
    workflow.add_edge("complete", END)
    
    return workflow.compile()

# ---------- 14. 主程序 ----------
if __name__ == "__main__":
    print("🚀 启动增强版报告生成系统...")
    print("✨ 新功能: 验证通过自动结束 | 自然章节过渡 | 智能图片插入")
    
    app = build_workflow()
    
    config = {
        "topic": "生成式 AI 如何重塑教育",
        "regenerate_count": 0,
        "current_report_version": 1,
        "output_format": "markdown",
        "section_histories": [],
        "report_versions": [],
        "search_results": [],
        "todo_list": [],
        "completed_tasks": []
    }
    
    try:
        # 设置递归限制
        result = app.invoke(config, config={"recursion_limit": 50})
        
        print("\n" + "="*50)
        print("📊 报告生成完成!")
        print("="*50)
        
        # 显示版本信息
        current_version = result.get('current_report_version', 1)
        print(f"\n📋 最终版本: v{current_version}")
        
        print("\n📁 生成的文件:")
        print(f"  - 最终报告: reports/report_v{current_version}_*.md")
        print(f"  - 章节草稿: drafts/section_*_v*_*.md")
        print(f"  - 版本历史: versions/v*/")
        print(f"  - 图片信息: images/section_*_v*_images.json")
        print(f"  - 任务列表: todo_v{current_version}.txt")
        
        validation = result.get('validation_result', {})
        if validation:
            print(f"\n✅ 最终得分: {validation.get('score', 0)}/10")
            print(f"📊 状态: {'通过' if validation.get('is_valid', False) else '需要改进'}")
            
            # 显示详细得分
            detailed_scores = validation.get('detailed_scores', {})
            if detailed_scores:
                print(f"📋 详细评分:")
                for aspect, score in detailed_scores.items():
                    print(f"  - {aspect}: {score}/10")
        
        # 显示章节版本信息
        print(f"\n📖 章节版本:")
        for history in sorted(result.get('section_histories', []), key=lambda h: h.section_index):
            current = history.get_current()
            if current:
                img_count = len(current.images)
                img_text = f" (含 {img_count} 图)" if img_count > 0 else ""
                print(f"  {history.section_index + 1}. {current.title} (v{current.version}){img_text}")
        
        # 显示过渡信息
        report_versions = result.get('report_versions', [])
        if report_versions:
            latest_version = report_versions[-1]
            if latest_version.transitions:
                print(f"\n🔗 章节过渡:")
                for transition in latest_version.transitions:
                    print(f"  {transition.from_section + 1} → {transition.to_section + 1}: {transition.connection_type}")
        
        print(f"\n📝 最终报告预览:")
        print("-" * 30)
        print(result["final_report"][:500] + "...")
        
    except Exception as e:
        print(f"❌ 生成失败: {e}")
        import traceback
        traceback.print_exc()

🚀 启动版本控制报告生成系统...
📋 初始计划制定...
📋 分发任务到 8 个worker
🔄 处理章节 1: 引言：生成式 AI 与教育的融合
🔄 处理章节 2: 生成式 AI 在教学中的应用
🔄 处理章节 3: 生成式 AI 对学生学习的影响
🔄 处理章节 4: AI 驱动的教育工具与平台
🔄 处理章节 5: 伦理与隐私问题
🔄 处理章节 6: 生成式 AI 对教育公平性的影响
🔄 处理章节 7: 未来趋势与展望
🔄 处理章节 8: 总结与建议
✅ 新版本: 未来趋势与展望 (v1)
✅ 章节 7 已更新到 v1
✅ 新版本: 引言：生成式 AI 与教育的融合 (v1)
✅ 新版本: 生成式 AI 对教育公平性的影响 (v1)
✅ 新版本: 生成式 AI 对学生学习的影响 (v1)
✅ 章节 1 已更新到 v1
✅ 新版本: 伦理与隐私问题 (v1)
✅ 章节 6 已更新到 v1
✅ 章节 3 已更新到 v1
✅ 章节 5 已更新到 v1
✅ 新版本: AI 驱动的教育工具与平台 (v1)
✅ 章节 4 已更新到 v1
✅ 新版本: 总结与建议 (v1)
✅ 章节 8 已更新到 v1
✅ 新版本: 生成式 AI 在教学中的应用 (v1)
✅ 章节 2 已更新到 v1
📝 合并报告 v1 (重新生成次数: 0)


/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/1125648981.py:143: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json.dump(history.dict(), f, ensure_ascii=False, indent=2)


🔍 验证报告 v1...
📊 验证结果: 得分 8.5/10 | 通过

📊 报告生成完成!

📋 最终版本: v1

📁 生成的文件:
  - 最终报告: reports/report_v1_*.md
  - 章节草稿: drafts/section_*_v*_*.md
  - 版本历史: versions/v*/
  - 任务列表: todo_v1.txt

✅ 最终得分: 8.5/10
📊 状态: 通过

📖 章节版本:
  1. 引言：生成式 AI 与教育的融合 (v1)
  2. 生成式 AI 在教学中的应用 (v1)
  3. 生成式 AI 对学生学习的影响 (v1)
  4. AI 驱动的教育工具与平台 (v1)
  5. 伦理与隐私问题 (v1)
  6. 生成式 AI 对教育公平性的影响 (v1)
  7. 未来趋势与展望 (v1)
  8. 总结与建议 (v1)

📝 最终报告预览:
------------------------------
# 生成式 AI 如何重塑教育

*版本: v1 | 生成时间: 2025-07-16 08:50:35*

## 版本信息

- 引言：生成式 AI 与教育的融合: v1
- 生成式 AI 在教学中的应用: v1
- 生成式 AI 对学生学习的影响: v1
- AI 驱动的教育工具与平台: v1
- 伦理与隐私问题: v1
- 生成式 AI 对教育公平性的影响: v1
- 未来趋势与展望: v1
- 总结与建议: v1

---

## 目录

1. [引言：生成式 AI 与教育的融合](#引言：生成式-ai-与教育的融合)
2. [生成式 AI 在教学中的应用](#生成式-ai-在教学中的应用)
3. [生成式 AI 对学生学习的影响](#生成式-ai-对学生学习的影响)
4. [AI 驱动的教育工具与平台](#ai-驱动的教育工具与平台)
5. [伦理与隐私问题](#伦理与隐私问题)
6. [生成式 AI 对教育公平性的影响](#生成式-ai-对教育公平性的影响)
7. [未来趋势与展望](#未来趋势与展望)
8. [总结与建议](#总结与建议)

---

## 引言：生成式 ...


/var/folders/z2/s5q05z8n4537pyvqd1m5yq9h0000gp/T/ipykernel_62815/1125648981.py:579: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "validation_result": validation.dict(),
